In [29]:
import os, sys
#import configparser

os.path.abspath("AzureDownload/config.txt")
os.getcwd()
#config = configparser.ConfigParser()
config.read("/home/jovyan/AzureDownload/config.txt")
config.sections()

['configuration', 'credentials']

In [42]:
## Required for Azure Data Lake Analytics job management
from azure.mgmt.datalake.analytics.job import DataLakeAnalyticsJobManagementClient
from azure.mgmt.datalake.analytics.job.models import JobInformation, JobState, USqlJobProperties
from azure.common.credentials import ServicePrincipalCredentials

credentials = ServicePrincipalCredentials(client_id =config.get("credentials","client"), secret=config.get("credentials","password"), tenant =config.get("credentials","tenant"))

subid= 'Microsoft Azure Sponsorship'
rg = 'IUNI-MAG-RG'
location = 'eastus2'
adla = 'iunimag'

adlaJobClient = DataLakeAnalyticsJobManagementClient(credentials, 'azuredatalakeanalytics.net')

In [50]:
script = open("/home/jovyan/openScience/code-data/usql-scripts/MAGget.usql", "r")

jobId = str(uuid.uuid4())
jobResult = adlaJobClient.job.create(
    adla,
    jobId,
    JobInformation(
        name='MAGget@jupyter',
        type='USql',
        degree_of_parallelism=15,
        properties=USqlJobProperties(script=script.read() )
    )
)

In [51]:
import time
from azure.storage.blob import BlockBlobService

CONTAINERNAME ='mag-2019-03-02'
BLOBNAME= "Open.csv"
LOCALFILENAME= "/home/jovyan/openScience/code-data/Open.csv"

#download from blob
block_blob_service=BlockBlobService(account_name=config.get("configuration","account"),account_key=config.get("configuration","password"))
#download from blob
t1=time.time()
block_blob_service.get_blob_to_path(CONTAINERNAME,BLOBNAME,LOCALFILENAME)
t2=time.time()
print(("It takes %s seconds to download "+BLOBNAME) % (t2 - t1))

It takes 1.821382761001587 seconds to download Open.csv


In [52]:
BLOBNAME= "Reproduce.csv"
LOCALFILENAME= "/home/jovyan/openScience/code-data/Reproduce.csv"

#download from blob
t1=time.time()
block_blob_service.get_blob_to_path(CONTAINERNAME,BLOBNAME,LOCALFILENAME)
t2=time.time()
print(("It takes %s seconds to download "+BLOBNAME) % (t2 - t1))

It takes 8.286332368850708 seconds to download Reproduce.csv


In [53]:
import time
from azure.storage.blob import BlockBlobService

CONTAINERNAME ='mag-2019-03-02'
BLOBNAME= "Open-old.csv"
LOCALFILENAME= "/home/jovyan/openScience/code-data/Open-old.csv"

#download from blob
block_blob_service=BlockBlobService(account_name=config.get("configuration","account"),account_key=config.get("configuration","password"))
#download from blob
t1=time.time()
block_blob_service.get_blob_to_path(CONTAINERNAME,BLOBNAME,LOCALFILENAME)
t2=time.time()
print(("It takes %s seconds to download "+BLOBNAME) % (t2 - t1))

It takes 1.7508304119110107 seconds to download Open-old.csv


In [54]:
BLOBNAME= "Reproduce-old.csv"
LOCALFILENAME= "/home/jovyan/openScience/code-data/Reproduce-old.csv"

#download from blob
t1=time.time()
block_blob_service.get_blob_to_path(CONTAINERNAME,BLOBNAME,LOCALFILENAME)
t2=time.time()
print(("It takes %s seconds to download "+BLOBNAME) % (t2 - t1))

It takes 0.8343608379364014 seconds to download Reproduce-old.csv
